In [1]:
import requests
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium import Choropleth

# Define the endpoint URL for the OSM API
endpoint = 'https://nominatim.openstreetmap.org/search'

# Define the parameters for the API request
params = {
    'q': 'schools in Nampula',
    'format': 'json',
    'limit': 30,
    'radius': 25000,
}

# Send the API request and get the response
response = requests.get(endpoint, params=params)

# Check if the request was successful (200 status code)
if response.status_code == 200:
    # Convert the response data to JSON format
    data = response.json()

    # Extract the relevant fields from the JSON data
    records = [(d['display_name'], d['lat'], d['lon']) for d in data]

    # Create a DataFrame from the records
    df = pd.DataFrame.from_records(records, columns=['Name', 'Latitude', 'Longitude'])

    # Split the 'Name' column into 'Name' and 'Address' columns based on the first comma
    df[['Name', 'Address']] = df['Name'].str.split(',', n=1, expand=True)

    # Remove any leading or trailing whitespace from the 'Name' and 'Address' columns
    df['Name'] = df['Name'].str.strip()
    df['Address'] = df['Address'].str.strip()

    display(df)

    # Create a map centered on Pretoria
    m = folium.Map(location=[-15.1207661, 39.2684548512737], zoom_start=11)

    # Add a choropleth layer to the map based on the number of schools in each suburb
    choropleth = Choropleth(
        geo_data='https://raw.githubusercontent.com/johan/world.geo.json/master/countries/ZAF.geo.json',
        name='choropleth',
        data=df.groupby('Address').size().reset_index(name='count'),
        columns=['Address', 'count'],
        key_on='feature.properties.name',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Number of Schools'
    ).add_to(m)

    # Add markers for each school to the map
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name']
        ).add_to(m)

    # Add the layer control
    folium.LayerControl().add_to(m)

    # Display the map
    display(m)

ModuleNotFoundError: No module named 'requests'